In [1]:
"""tr_letters= 'A A B A B C A B C'.split()
tr_labels= [1, 4, 2, 0, 0, 6, 3, 4, 1]

train['letter']= tr_letters
train['label']= tr_labels

def Nan_C(row):
    if row['letter']== 'C':
        row['letter_encoded_mean']= np.nan
    return  row
train= train.apply(lambda row: Nan_C(row), axis=1) """

import pandas as pd
import numpy as np

from sklearn  import base
from sklearn.model_selection import train_test_split, KFold

from termcolor import colored

In [2]:
########################### Creo Training Set ##############################
train= pd.DataFrame()

tr_letters= 'A B A B A C'.split()
tr_labels= [1, 4, 2, 0, 0, 6]

train['letter']= tr_letters
train['label']= tr_labels

#Mi creo una copia
copy_train= train.copy()

################ Creo Test: ci metto una categoria nota, A, e un'altra non vista nel Train, D ########################
test_letters= 'A D'.split()
test_labels= [10, 8]

test= pd.DataFrame()
test['letter']= test_letters
test['label']= test_labels

#Mi creo una copia
copy_test= test.copy()


train

,letter,label
0,A,1
1,B,4
2,A,2
3,B,0
4,A,0
5,C,6


In [3]:
"""########################### Creo Training Set 2 ##############################
train= pd.DataFrame()

tr_letters= 'A B C A A A'.split()
tr_labels= [10, 4, 2, 0, 0, 6]

train['letter']= tr_letters
train['label']= tr_labels"""

"########################### Creo Training Set 2 ##############################\ntrain= pd.DataFrame()\n\ntr_letters= 'A B C A A A'.split()\ntr_labels= [10, 4, 2, 0, 0, 6]\n\ntrain['letter']= tr_letters\ntrain['label']= tr_labels"

In [4]:
test

,letter,label
0,A,10
1,D,8


## <font color = 'orange' > Faccio Encoding Sul Train

In [5]:
kf = KFold(n_splits = 3, shuffle = False, random_state= 0)

name_encoded_mean= train.columns[0] + '_encoded_mean'
name_encoded_std= train.columns[0] + '_encoded_std'

column_to_encode= 'letter'
target_column= 'label'

################################### INIZIA CICLO ################################
threshold= 0
count= 1
show_steps= True

print('\033[1m\tINITIAL TRAIN \n {} \n \033[0m'.format(train) )
print(colored('\033[1m Threshold scelta: {}'.format(threshold), 'red') )
for tr_ind, val_ind in kf.split(train):
    
    if show_steps== True:
        continua= str(input('Continuo il ciclo? y/n \n'))
        if continua.lower()[0]!= 'y':
            print('Stoppo il ciclo')
            break
    
    
    
    print('\t\033[1mITERATION K-FOLD {}\033[0m \nIndici delle Train-Folds: {}, \033[1m Righe (Test Fold) da encodare: {}\033[0m \n'.format(count, tr_ind, colored(val_ind, 'red')))  #sono gli indici delle folds
    
    X_tr, X_val = train.iloc[tr_ind], train.iloc[val_ind]
    
    diz_mean= X_tr.groupby(column_to_encode)[target_column].mean().to_dict()
    diz_std= X_tr.groupby(column_to_encode)[target_column].std().to_dict()
    
    #### Tratto come Nan chi non supera la threshold, mentre per gli altri faccio Encoding classico
    frequenze= X_tr.groupby('letter').size()
    cat_oltre_threshold= frequenze[frequenze >= threshold].index
    print(cat_oltre_threshold)
    
    train.loc[train.index[val_ind], name_encoded_mean] = X_val[column_to_encode].apply(lambda x: diz_mean[x] if x in cat_oltre_threshold else np.nan )
    train.loc[train.index[val_ind], name_encoded_std] = X_val[column_to_encode].apply(lambda x: diz_std[x] if x in cat_oltre_threshold else np.nan)
    
    
    mean_of_target = X_tr[target_column].mean()  #media di tutto il target sul KFold Train, not grouped
    std_of_target= X_tr[target_column].std()    # # std di tutto il target sul KFold Train, not grouped
    
    if show_steps== True:
        print("\033[1m media e std del target sulle K-1 Folds sono {} e {} \033[0m".format(mean_of_target, std_of_target))
        print('\033[1m', colored(train, 'blue'))
    
    #train[name_encoded_mean].fillna(mean_of_target, inplace = True) #ai NaN metto la media del KFold Train Target
    #train[name_encoded_std].fillna(std_of_target, inplace = True)   # ai NaN metto la std dev media del Kfold Train Target
    
    count+=1
    
train

	INITIAL TRAIN 
   letter  label
0      A      1
1      B      4
2      A      2
3      B      0
4      A      0
5      C      6 
 
 Threshold scelta: 0
Continuo il ciclo? y/n 
y
	ITERATION K-FOLD 1 
Indici delle Train-Folds: [2 3 4 5],  Righe (Test Fold) da encodare: [0 1] 

Index(['A', 'B', 'C'], dtype='object', name='letter')
 media e std del target sulle K-1 Folds sono 2.0 e 2.8284271247461903 
   letter  label  letter_encoded_mean  letter_encoded_std
0      A      1                  1.0            1.414214
1      B      4                  0.0                 NaN
2      A      2                  NaN                 NaN
3      B      0                  NaN                 NaN
4      A      0                  NaN                 NaN
5      C      6                  NaN                 NaN
Continuo il ciclo? y/n 
y
	ITERATION K-FOLD 2 
Indici delle Train-Folds: [0 1 4 5],  Righe (Test Fold) da encodare: [2 3] 

Index(['A', 'B', 'C'], dtype='object', name='letter')
 media e std del tar

,letter,label,letter_encoded_mean,letter_encoded_std
0,A,1,1.0,1.414214
1,B,4,0.0,NaN
2,A,2,0.5,0.707107
3,B,0,4.0,NaN
4,A,0,1.5,0.707107
5,C,6,NaN,NaN


In [6]:
test

,letter,label
0,A,10
1,D,8


#### NB: di default Python usa std campionaria (N-1 al denominatore.) Nella 1° iterazione, valori di A nelle due fold sono [0,2] e Sample std= 1.414

In [36]:
def std_dev(array):
    media= np.mean(array)
    return np.sqrt(np.mean((array - media)**2))

def std_dev_campionaria(array):
    media= np.mean(array)
    return np.sqrt(np.sum((array - media)**2)/ (len(array) -1))

print("Dato l'array {} \nla std dev (sigma) è {}, mentre S, quella campionaria (con N-1 al denominatore) è {}".format(np.array([2,0]), 
                                                    std_dev(np.array([2,0])), round(std_dev_campionaria(np.array([2,0])),3 ) ))

Dato l'array [2 0] 
la std dev (sigma) è 1.0, mentre S, quella campionaria (con N-1 al denominatore) è 1.414


## Fillo i Nan rispettivamente con media e std della popolazione del target

In [37]:
f= train.groupby('letter').size() 
f[f> 1]

letter
A    3
B    2
dtype: int64

In [38]:
kf = KFold(n_splits = 3, shuffle = False, random_state= 0)

name_encoded_mean= train.columns[0] + '_encoded_mean'
name_encoded_std= train.columns[0] + '_encoded_std'

column_to_encode= 'letter'
target_column= 'label'

threshold= 1
count= 1


for tr_ind, val_ind in kf.split(train):
    print('\tITERATION K-FOLD {} \nIndici delle Train-Folds: {}, \033[1mTest Fold da fillare: {}\033[0m \n'.format(count, tr_ind, val_ind))  #sono gli indici delle folds
    
    X_tr, X_val = train.iloc[tr_ind], train.iloc[val_ind]
    
    diz_mean= X_tr.groupby(column_to_encode)[target_column].mean().to_dict()
    diz_std= X_tr.groupby(column_to_encode)[target_column].std().to_dict()
    
     #### Tratto come Nan chi non supera la threshold, mentre per gli altri faccio Encoding classico
    frequenze= X_tr.groupby('letter').size()
    cat_oltre_threshold= frequenze[frequenze >= threshold].index
    print(cat_oltre_threshold)
    
    train.loc[train.index[val_ind], name_encoded_mean] = X_val[column_to_encode].apply(lambda x: diz_mean[x] if x in cat_oltre_threshold else np.nan )
    train.loc[train.index[val_ind], name_encoded_std] = X_val[column_to_encode].apply(lambda x: diz_std[x] if x in cat_oltre_threshold else np.nan)
    
    
    mean_of_target = X_tr[target_column].mean()  #media di tutto il target sul KFold Train, not grouped
    std_of_target= X_tr[target_column].std()    # # std di tutto il target sul KFold Train, not grouped
    
    
    """ RISPETTO AL BLOCCO DI PRIMA AGGIUNGO SOLO QUESTO, VA MESSO NEL CICLO PERCHè è DINAMICO """
    train[name_encoded_mean].fillna(mean_of_target, inplace = True) #ai NaN metto la media del KFold Train Target
    train[name_encoded_std].fillna(std_of_target, inplace = True)   # ai NaN metto la std dev media del Kfold Train Target
    
    count+=1
    
train

	ITERATION K-FOLD 1 
Indici delle Train-Folds: [2 3 4 5], Test Fold da fillare: [0 1] 

Index(['A', 'B', 'C'], dtype='object', name='letter')
	ITERATION K-FOLD 2 
Indici delle Train-Folds: [0 1 4 5], Test Fold da fillare: [2 3] 

Index(['A', 'B', 'C'], dtype='object', name='letter')
	ITERATION K-FOLD 3 
Indici delle Train-Folds: [0 1 2 3], Test Fold da fillare: [4 5] 

Index(['A', 'B'], dtype='object', name='letter')


,letter,label,letter_encoded_mean,letter_encoded_std
0,A,1,1.00,1.414214
1,B,4,0.00,2.828427
2,A,2,0.50,0.707107
3,B,0,4.00,2.753785
4,A,0,1.50,0.707107
5,C,6,1.75,1.707825


In [39]:
train.iloc[:4]['label'].describe().head(3)

count    4.000000
mean     1.750000
std      1.707825
Name: label, dtype: float64

## <font color = 'orange' > Ora Encoding sul Test

In [40]:
test

,letter,label
0,A,10
1,D,8


In [41]:
train.groupby('letter').mean()

,label,letter_encoded_mean,letter_encoded_std
letter,,,
A,1,1.00,0.942809
B,2,2.00,2.791106
C,6,1.75,1.707825


In [42]:
test[name_encoded_mean]= test['letter'].map(train.groupby('letter').mean()['letter_encoded_mean']) #mappo categorie con le loro medie sul train
test[name_encoded_std]= test['letter'].map(train.groupby('letter').mean()['letter_encoded_std']) #metto std_dev media sul train
test

,letter,label,letter_encoded_mean,letter_encoded_std
0,A,10,1.0,0.942809
1,D,8,NaN,NaN


## Fillo mean e std di nuove categorie nel Test con le rispettive medie nel Train

In [43]:
train

,letter,label,letter_encoded_mean,letter_encoded_std
0,A,1,1.00,1.414214
1,B,4,0.00,2.828427
2,A,2,0.50,0.707107
3,B,0,4.00,2.753785
4,A,0,1.50,0.707107
5,C,6,1.75,1.707825


In [44]:
mean_letter_encoded_tr= train['letter_encoded_mean'].mean()
std_letter_encoded_tr= train['letter_encoded_std'].mean()

test['letter_encoded_mean'].fillna(mean_letter_encoded_tr, inplace = True) #ai NaN metto la media di encoding_mean
test['letter_encoded_std'].fillna(std_letter_encoded_tr, inplace = True)   #ai NaN metto la media di encoding_std

test

,letter,label,letter_encoded_mean,letter_encoded_std
0,A,10,1.000000,0.942809
1,D,8,1.458333,1.686411


# <font color = 'orange'> Final Step, aggiungo a Train e Test il Noise proporzionale a std dev 
 <font color = 'orange'> <h4>Lo ottengo come Gaussiana centrata sul Mean_encoding avente std pari a 0.1*Std_Encoding </h6>

In [45]:
np.random.seed(2)  #fisso un random seed

moltiplicatore= 0.01

train['ENCODED WITH NOISE']= np.random.normal(train['letter_encoded_mean'], moltiplicatore*train['letter_encoded_std'],
                                              size= len(train))

train

,letter,label,letter_encoded_mean,letter_encoded_std,ENCODED WITH NOISE
0,A,1,1.00,1.414214,0.994106
1,B,4,0.00,2.828427,-0.001591
2,A,2,0.50,0.707107,0.484895
3,B,0,4.00,2.753785,4.045170
4,A,0,1.50,0.707107,1.487318
5,C,6,1.75,1.707825,1.735624


In [46]:
np.random.normal(1, 0.1*1.414)

1.0711074323861474

In [292]:
test['ENCODED WITH NOISE']= np.random.normal(test['letter_encoded_mean'], moltiplicatore*test['letter_encoded_std'], size= len(test))

test

,letter,label,letter_encoded_mean,letter_encoded_std,ENCODED WITH NOISE
0,A,10,1.000000,0.942809,0.882593
1,D,8,1.458333,1.686411,1.279919


In [293]:
######################################################################################################################

# <center> <font color = 'red' > Generalizzo tutto con una classe

In [47]:
class KFoldTargetEncoderTrain_std(base.BaseEstimator, base.TransformerMixin):

    def __init__(self, colnames,targetName,n_fold=5,verbosity=True,discardOriginal_col=False, threshold= 1):

        self.colnames = colnames
        self.targetName = targetName
        self.n_fold = n_fold
        self.verbosity = verbosity
        self.discardOriginal_col = discardOriginal_col
        self.threshold= threshold

    def fit(self, X, y=None):
        return self

    def transform(self,X):

        assert(type(self.targetName) == str)
        assert(type(self.colnames) == str)
        assert(self.colnames in X.columns)
        assert(self.targetName in X.columns)
       
        kf = KFold(n_splits = self.n_fold, shuffle = False, random_state=2019)

        col_mean_name = self.colnames + '_' + 'enc_mean'
        col_std_name= self.colnames + '_' + 'enc_std'
        
        #Le inizializzo con tutti NaN
        X[col_mean_name] = np.nan
        X[col_std_name] = np.nan
        
        #count= 1
        #A ogni giro (per ogni K-Fold) fillo valori della Test Fold con la media delle Train Fold.
        for tr_ind, val_ind in kf.split(X):
            print('Indici delle Train-Folds: {},\033[1mTest Fold da fillare: {}\033[0m \n'.format(tr_ind, val_ind))  #sono gli indici delle folds

            X_tr, X_val = X.iloc[tr_ind], X.iloc[val_ind]

            diz_mean= X_tr.groupby(self.colnames)[self.targetName].mean().to_dict()
            diz_std= X_tr.groupby(self.colnames)[self.targetName].std().to_dict()

            #### Tratto come Nan chi non supera la threshold, agli altri assegno media e std sulle altre folds
            frequenze= X_tr.groupby(self.colnames).size()
            cat_oltre_threshold= frequenze[frequenze >= self.threshold].index
            print('cat_oltre_threshold', cat_oltre_threshold)

            X.loc[X.index[val_ind], col_mean_name] = X_val[self.colnames].apply(lambda x: diz_mean[x] if x in cat_oltre_threshold else np.nan )
            X.loc[X.index[val_ind], col_std_name] = X_val[self.colnames].apply(lambda x: diz_std[x] if x in cat_oltre_threshold else np.nan)

            mean_of_target = X_tr[self.targetName].mean()  #media di tutto il target sul KFold Train, not grouped
            std_of_target= X_tr[self.targetName].std()    # # std di tutto il target sul KFold Train, not grouped

            X[col_mean_name].fillna(mean_of_target, inplace = True) #ai NaN metto la media del KFold Train Target
            X[col_std_name].fillna(std_of_target, inplace = True)   # ai NaN metto la std dev media del Kfold Train Target

            #count+=1

        if self.verbosity:

            encoded_feature = X[col_mean_name].values
            print('Correlation between the new feature, {} and, {} is {}.'.format(col_mean_name,
                                    self.targetName,np.corrcoef(X[self.targetName].values, encoded_feature)[0][1]))
            
        if self.discardOriginal_col:
            X = X.drop(self.targetName, axis=1)
            
        return X
    
    
    
class KFoldTargetEncoderTest_std(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self,train,colNames,Train_encoded_mean, Train_encoded_std):
        
        self.train = train
        self.colNames = colNames
        self.Train_encoded_mean = Train_encoded_mean
        self.Train_encoded_std = Train_encoded_std
        
    def fit(self, X, y=None):
        return self

    def transform(self,X):

        ### Media e std del Target grouped per categoria su tutto il Train
        category_mean_tr = self.train[[self.colNames,self.Train_encoded_mean]].groupby(self.colNames).mean() #.reset_index() 
        category_std_tr= self.train[[self.colNames,self.Train_encoded_std]].groupby(self.colNames).mean() #.reset_index() 
        
        ### Media dell'Encoding basato su mean e std sul Train (non grouped per categoria)
        population_mean_tr= self.train[self.Train_encoded_mean].mean()
        population_std_tr= self.train[self.Train_encoded_std].mean()
        
        
        ### Metto nel Test i valori medi della mean e std del Train per quella categoria
        X[self.Train_encoded_mean]= X[self.colNames].map(category_mean_tr.squeeze().to_dict())
        X[self.Train_encoded_std]= X[self.colNames].map(category_std_tr.squeeze().to_dict())
        
        ### Fillo i Missing Values (Categorie Nuove) con i valori medi di mean e std (encodate) di tutte la variabile nel Train
        print('Filled {} Missing Values with Average mean and std equals to {}, {}'.format(
                                                        np.sum(np.sum(X.isna())), population_mean_tr , population_std_tr))
        
        X[self.Train_encoded_mean].fillna(population_mean_tr, inplace = True) #fillo con media di enc_mean sul Train
        X[self.Train_encoded_std].fillna(population_std_tr, inplace = True)   #fillo con media di enc_std sul Train
        
        return X

##### Verifico che ottengo stesso risultato sul train usando la Classe

In [48]:
########################### Creo Training Set ##############################
copy_train= pd.DataFrame()

tr_letters= 'A B A B A C'.split()
tr_labels= [1, 4, 2, 0, 0, 6]

copy_train['letter']= tr_letters
copy_train['label']= tr_labels


################ Creo Test: ci metto una categoria nota, A, e un'altra non vista nel Train, D ########################
test_letters= 'A D'.split()
test_labels= [10, 8]

copy_test= pd.DataFrame()
copy_test['letter']= test_letters
copy_test['label']= test_labels

copy_train

,letter,label
0,A,1
1,B,4
2,A,2
3,B,0
4,A,0
5,C,6


In [49]:
copy_train

,letter,label
0,A,1
1,B,4
2,A,2
3,B,0
4,A,0
5,C,6


In [50]:
targetc = KFoldTargetEncoderTrain_std('letter', 'label' , n_fold= 3)
copy_train_enc = targetc.fit_transform(copy_train)
copy_train_enc

Indici delle Train-Folds: [2 3 4 5],Test Fold da fillare: [0 1] 

cat_oltre_threshold Index(['A', 'B', 'C'], dtype='object', name='letter')
Indici delle Train-Folds: [0 1 4 5],Test Fold da fillare: [2 3] 

cat_oltre_threshold Index(['A', 'B', 'C'], dtype='object', name='letter')
Indici delle Train-Folds: [0 1 2 3],Test Fold da fillare: [4 5] 

cat_oltre_threshold Index(['A', 'B'], dtype='object', name='letter')
Correlation between the new feature, letter_enc_mean and, label is -0.3841614968388928.


,letter,label,letter_enc_mean,letter_enc_std
0,A,1,1.00,1.414214
1,B,4,0.00,2.828427
2,A,2,0.50,0.707107
3,B,0,4.00,2.753785
4,A,0,1.50,0.707107
5,C,6,1.75,1.707825


In [51]:
train

,letter,label,letter_encoded_mean,letter_encoded_std,ENCODED WITH NOISE
0,A,1,1.00,1.414214,0.994106
1,B,4,0.00,2.828427,-0.001591
2,A,2,0.50,0.707107,0.484895
3,B,0,4.00,2.753785,4.045170
4,A,0,1.50,0.707107,1.487318
5,C,6,1.75,1.707825,1.735624


### Corretto, ho ottenuto lo stesso di prima. Ora verifico per il Test

In [274]:
copy_train_enc.mean()

label              2.166667
letter_enc_mean    1.458333
letter_enc_std     1.686411
dtype: float64

In [275]:
test_targetc = KFoldTargetEncoderTest_std(copy_train_enc, 'letter', 'letter_enc_mean', 'letter_enc_std')
copy_test_enc= test_targetc.fit_transform(copy_test)
copy_test_enc

Filled 2 Missing Values with Average mean and std equals to 1.4583333333333333, 1.6864107751325605


,letter,label,letter_enc_mean,letter_enc_std
0,A,10,1.000000,0.942809
1,D,8,1.458333,1.686411


In [276]:
test

,letter,label
0,A,10
1,D,8


### Corretto anche questo!! :)

In [ ]:
################################### ALTRO CASO ESTREMO DA GESTIRE ###############################################

In [66]:
prova_train= pd.read_csv('prova.csv').iloc[: ,1:]
prova_train

,label,letter
0,0,A
1,4,A
2,1,B
3,1,B
4,1,A
5,5,A
6,0,B
7,6,B


In [67]:
for col in prova_train.drop('label', axis=1).columns:
    targetc = KFoldTargetEncoderTrain_std(col, 'label' ,n_fold= 4, discardOriginal_col= False)
    prova_train = targetc.fit_transform(prova_train)
    
prova_train

Indici delle Train-Folds: [2 3 4 5 6 7], Test Fold da fillare: [0 1] 

Indici delle Train-Folds: [0 1 4 5 6 7], Test Fold da fillare: [2 3] 

Indici delle Train-Folds: [0 1 2 3 6 7], Test Fold da fillare: [4 5] 

Indici delle Train-Folds: [0 1 2 3 4 5], Test Fold da fillare: [6 7] 

Correlation between the new feature, letter_enc_mean and, label is -0.30530408919874125.


,label,letter,letter_enc_mean,letter_enc_std
0,0,A,3.0,2.828427
1,4,A,3.0,2.828427
2,1,B,3.0,4.242641
3,1,B,3.0,4.242641
4,1,A,2.0,2.828427
5,5,A,2.0,2.828427
6,0,B,1.0,0.000000
7,6,B,1.0,0.000000


#### <font color='blue'> DOVE HO STD DEV 0 (perchè nel K-Fold Train ho tutti valori uguali per quella categoria), AVRO ZERO NOISE; METTO DI DEFAULT STD A 0.01 OGNI VOLTA CHE SUCCEDE COSI